# PCVI Calibration

### Christopher Rapp

rapp5@purdue.edu

---

**SECTION:** Import Libraries

In [1]:
import os
import glob
import re
import numpy as np
import pandas as pd
from datetime import datetime
from numpy import linspace
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from scipy.optimize import curve_fit


**SECTION:** Define working directories

In [2]:
# Location of files necessary
import_OPS = '/Users/christopherrapp/Documents/Purdue/spider/PCVI_calibration/import/OPS/csv_files'

# Location of files necessary
import_APS = '/Users/christopherrapp/Documents/Purdue/spider/PCVI_calibration/import/APS'

# Location to send results
export_results = '/Users/christopherrapp/Documents/Purdue/spider/PCVI_calibration/export/pcvi/'

# Location to send plots
export_plots = '/Users/christopherrapp/Documents/Purdue/spider/PCVI_calibration/plots/pcvi/'


**SECTION:** List directories

In [3]:
# List directories in cwd
dirs = os.listdir(import_OPS)

# Loop through directories and identify those that match a regex date string
tmp = []
for i in range(0, len(dirs), 1):
    x = re.findall(r'\d{4}\w+\d{2}', dirs[i])
    tmp.append(x)

# Create a list of the matches    
directories = list(filter(len, tmp))

# Print to screen the available directories
print("Available Directories", directories)


Available Directories [['2021June16'], ['2021June09'], ['2021June08'], ['2021June15'], ['2021June14']]


In [4]:
D = {}
F = {}
tmp = []
for i in (range(0, len(directories), 1)):
    
    # Specify which day of data
    x = str(directories[i][0])

    # Define paths
    inlet_path = import_OPS + '/' + x + '/inlet/'
    outlet_path = import_OPS + '/' + x + '/outlet/'

    # List the files in paths
    inlet_files = os.listdir(inlet_path)
    outlet_files = os.listdir(outlet_path)
    
    # Initialize dictionary
    file_information = {
        'Inlet FilePath': [],
        'Inlet File': [],
        'Outlet FilePath': [],
        'Outlet File': [],
        'Inlet PF': [],
        'Inlet AF': [],
        'Inlet BF': [],
        'Outlet PF': [],
        'Outlet AF': []
    }

    # Extract file information from the filenames in the directory
    for j in range(0, len(inlet_files), 1):
        
        file_information["Inlet FilePath"].append(inlet_path + inlet_files[j])
        file_information["Inlet File"].append(inlet_files[j])

        # Extract the necessary information from filenames
        in_PF = str(re.findall(r'PF\d+[_]?\d*', inlet_files[j]))
        in_AF = str(re.findall(r'AF\d+[_]?\d*', inlet_files[j]))
        in_BF = str(re.findall(r'BF\d+[_]?\d*', inlet_files[j]))
        
        in_PF = float(in_PF.replace('_', '.').replace('PF', '').replace("['", '').replace("']", ''))
        in_AF = float(in_AF.replace('_', '.').replace('AF', '').replace("['", '').replace("']", ''))
        in_BF = float(in_BF.replace('_', '.').replace('BF', '').replace("['", '').replace("']", ''))
        
        file_information["Inlet PF"].append(in_PF)
        file_information["Inlet AF"].append(in_AF)
        file_information["Inlet BF"].append(in_BF)
        
    # Extract file information from the filenames in the directory
    for k in range(0, len(outlet_files), 1):

        file_information["Outlet FilePath"].append(outlet_path + outlet_files[k])
        file_information["Outlet File"].append(outlet_files[k])

        # Extract the necessary information from filenames
        out_PF = str(re.findall(r'PF\d+[_]?\d*', outlet_files[k]))
        out_AF = str(re.findall(r'AF\d+[_]?\d*', outlet_files[k]))
        
        out_PF = float(out_PF.replace('_', '.').replace('PF', '').replace("['", '').replace("']", ''))
        out_AF = float(out_AF.replace('_', '.').replace('AF', '').replace("['", '').replace("']", ''))
        
        file_information["Outlet PF"].append(out_PF)
        file_information["Outlet AF"].append(out_AF)

    # Dataframe conversion
    file_info_df = pd.DataFrame.from_dict(file_information, orient = 'index')
    D[i] = file_info_df.transpose()
    
    # Seperate data
    IN = D[i].iloc[:, [0,1,4,5,6]].copy(deep = True)
    IN['AF'] = IN.loc[:, ('Inlet AF')]
    IN['PF'] = IN.loc[:, ('Inlet PF')]

    OUT = D[i].iloc[:, [2,3,7,8]].copy(deep = True)
    OUT['AF'] = OUT.loc[:, ('Outlet AF')]
    OUT['PF'] = OUT.loc[:, ('Outlet PF')]

    # Merge on AF as that is the indicator they are similarly tested files as PF is the same across both
    F[i] = pd.merge(IN, OUT, how = "inner", on = ("AF", "PF"))
    

In [5]:
# View which day of data you are interested in
F[1]


,Inlet FilePath,Inlet File,Inlet PF,Inlet AF,Inlet BF,AF,PF,Outlet FilePath,Outlet File,Outlet PF,Outlet AF
0,/Users/christopherrapp/Documents/Purdue/spider...,2021June09 PF5 AF2_2 BF2_8 inlet.csv,5,2.2,2.8,2.2,5,/Users/christopherrapp/Documents/Purdue/spider...,2021June09 PF5 AF2_2 outlet.csv,5,2.2
1,/Users/christopherrapp/Documents/Purdue/spider...,2021June09 PF5 AF1_7 BF3_3 inlet.csv,5,1.7,3.3,1.7,5,/Users/christopherrapp/Documents/Purdue/spider...,2021June09 PF5 AF1_7 outlet.csv,5,1.7
2,/Users/christopherrapp/Documents/Purdue/spider...,2021June09 PF5 AF1_6 BF3_4 inlet.csv,5,1.6,3.4,1.6,5,/Users/christopherrapp/Documents/Purdue/spider...,2021June09 PF5 AF1_6 outlet.csv,5,1.6
3,/Users/christopherrapp/Documents/Purdue/spider...,2021June09 PF5 AF2 BF3 inlet.csv,5,2,3,2,5,/Users/christopherrapp/Documents/Purdue/spider...,2021June09 PF5 AF2 outlet.csv,5,2
4,/Users/christopherrapp/Documents/Purdue/spider...,2021June09 PF5 AF1_8 BF3_2 inlet.csv,5,1.8,3.2,1.8,5,/Users/christopherrapp/Documents/Purdue/spider...,2021June09 PF5 AF1_8 outlet.csv,5,1.8
5,/Users/christopherrapp/Documents/Purdue/spider...,2021June09 PF5 AF1_9 BF3_1 inlet.csv,5,1.9,3.1,1.9,5,/Users/christopherrapp/Documents/Purdue/spider...,2021June09 PF5 AF1_9 outlet.csv,5,1.9
6,/Users/christopherrapp/Documents/Purdue/spider...,2021June09 PF5 AF2_1 BF2_9 inlet.csv,5,2.1,2.9,2.1,5,/Users/christopherrapp/Documents/Purdue/spider...,2021June09 PF5 AF2_1 outlet.csv,5,2.1
7,/Users/christopherrapp/Documents/Purdue/spider...,2021June09 PF5 AF1_5 BF3_5 inlet.csv,5,1.5,3.5,1.5,5,/Users/christopherrapp/Documents/Purdue/spider...,2021June09 PF5 AF1_5 outlet.csv,5,1.5
8,/Users/christopherrapp/Documents/Purdue/spider...,2021June09 PF5 AF2_3 BF2_7 inlet.csv,5,2.3,2.7,2.3,5,/Users/christopherrapp/Documents/Purdue/spider...,2021June09 PF5 AF2_3 outlet.csv,5,2.3


Transmission Efficiency: $$ \frac{Outlet\ Concentration\ cm^{-3}}{Inlet\ Concentration\ cm^{-3}} \cdot \frac{Sample\ Flow\ (L\cdot\min^{-1})}{Inlet\ Flow\ (L\cdot\min^{-1})} $$

Flow Equilibrium:

$$ PF = AF + BF $$

$$ IF = PF + SF - AF $$

$$ BF = IF - 1 $$


Logistic Curve (Sigmoid Curve):

$$ D(t)\ = \frac{L}{1+e^{-k(x-x_{0})}} $$

In [6]:
# Loop through dated directories for all matched inlet and outlet files
for i in range(0, len(F), 1):
    
    # Create a temporary variable of the file information dataframe
    X = F[i]
    
    for j in range(0, len(X), 1):
        
        # Calculate flows
        PF = pd.to_numeric(X["Inlet PF"][j])    # Pump flow
        AF = pd.to_numeric(X["AF"][j])          # Add flow
        SF = 1                                  # Sample flow
        IF = (PF + SF) - AF                     # Inlet flow 
        BF = IF - 1                             # Balance flow inlet flow - aerosol flow 1 lpm
        
        # Use pd.read_csv() to read in the OPS data
        OPS_IN = pd.read_csv(X["Inlet FilePath"][j], sep = ',', skip_blank_lines = True, skiprows = 26,
                             header = 0, decimal = '.', dtype = "float64")
        
        OPS_OUT = pd.read_csv(X["Outlet FilePath"][j], sep = ',', skip_blank_lines = True, skiprows = 26,
                             header = 0, decimal = '.', dtype = "float64")
        
        # Calculate the mean concentrations over the concentration columns
        OPS_IN["Inlet Mean Concentration"] = OPS_IN[OPS_IN.columns[pd.Series(OPS_IN.columns).str.startswith('Data')]].mean(axis = 1)
        OPS_IN["Diameter Midpoint"] = OPS_IN['MPDiam'].round(2)
        
        OPS_OUT["Outlet Mean Concentration"] = OPS_OUT[OPS_OUT.columns[pd.Series(OPS_OUT.columns).str.startswith('Data')]].mean(axis = 1)
        OPS_OUT["Diameter Midpoint"] = OPS_OUT['MPDiam'].round(2)
        
        # Formatting and adding columns
        data_df = OPS_IN[['Diameter Midpoint', 'Inlet Mean Concentration']].copy(deep = True)
        data_df['Outlet Mean Concentration'] = OPS_OUT["Outlet Mean Concentration"]
        data_df['PF'] = int(PF)
        data_df['AF'] = AF
        data_df['SF'] = SF
        data_df['IF'] = IF
        data_df['BF'] = round(BF, 1)
        
        # Calculate transmission efficiency
        data_df['Outlet/Inlet'] = data_df['Outlet Mean Concentration']/data_df['Inlet Mean Concentration']
        data_df['IF/SF'] = data_df['IF']/data_df['SF']
        data_df['Transmission Efficiency'] = data_df['Outlet/Inlet']/data_df['IF/SF']
        
        # Mask any inf values with NaN
        # These values will mess with any curve fitting
        data_df = data_df.mask(np.isinf(data_df))
        
        # Find the date information from the filename
        date = str(re.findall(r'\d{4}\w+\d{2}', X["Inlet File"][0])).replace("['", '').replace("']", '')
        
        # Format the date string
        date = datetime.strptime(date, "%Y%B%d")
        
        date = datetime.strftime(date, format = "%Y-%m-%d")
        
        file_label = date+"_PF"+str(data_df['PF'][0]).replace('.', '_')+"_AF"+str(data_df['AF'][0]).replace('.', '_')+"_BF"+str(data_df['BF'][0]).replace('.', '_')+"_results"+".csv"
        
        file_nm = export_results + file_label
        
        data_df.to_csv(file_nm)
        
        # Plotting size distributions
        # ---------------------------------------------- #
        
        # Set plotting style
        plt.style.use('ggplot')
        
        # Define figure axes and subplots
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (16,12), sharex = True)
        
        # Create a title string
        plot_title = "PCVI Size Distributions - " + date + ", PF = " + str(data_df["PF"][0]) + ", AF = " + str(data_df["AF"][0]) + ", SF = " + str(data_df["SF"][0]) + ", IF = " + str(data_df["IF"][0]) + ", IN/SF = " + str(data_df["IF/SF"][0])
        
        # Change font and type of super title
        fig.suptitle(plot_title, size = "x-large", weight = "demibold")

        # Indices for plot using list comprehension
        idx = np.asarray([i for i in range(len(data_df['Diameter Midpoint']))])

        # Define width
        width = 0.5

        # Create bars
        ax1.bar(x = idx, height = data_df['Outlet Mean Concentration'], width = width, align = "center")

        # Plot cosmetics
        ax1.set_xticks(idx)
        ax1.set_xticklabels(data_df['Diameter Midpoint'])
        ax1.set_xlim(left = data_df['Diameter Midpoint'][0])
        ax1.set_title("Outlet Size Distribution")
        ax1.set_ylabel(r'Outlet Mean Concentration $\mathrm{(cm^{-3})}$')

        # Indices for plot using list comprehension
        idx = np.asarray([i for i in range(len(data_df['Diameter Midpoint']))])

        # Define width
        width = 0.5

        # Create bars
        ax2.bar(x = idx, height = data_df['Inlet Mean Concentration'], width = width, align = "center")

        # Plot cosmetics
        ax2.set_xticks(idx)
        ax2.set_xticklabels(data_df['Diameter Midpoint'])
        ax2.set_xlim(left = data_df['Diameter Midpoint'][0])
        ax2.set_title("Inlet Size Distribution")
        ax2.set_xlabel(r'Median Midpoint Diameter $\mathrm{(\mu m)}$')
        ax2.set_ylabel(r'Inlet Mean Concentration $\mathrm{(cm^{-3})}$')

        # Create a figure name string
        figure_label = date+"_PF"+str(data_df['PF'][0]).replace('.', '_')+"_AF"+str(data_df['AF'][0]).replace('.', '_')+"_ops_size_distribution"+".png"

        figure_nm = export_plots + "size_distributions/" + figure_label

        # Save the figure
        plt.savefig(figure_nm)
        
        # Close the figure to prevent memory overflow
        plt.close()
        
        # Plotting transmission efficiencies
        # ---------------------------------------------- #
        
        # Remove NaN's to prevent errors in curve fitting
        data_df = data_df.dropna(thresh = 10)

        # Specify variables used in plotting
        xdata = data_df["Diameter Midpoint"]
        xdata_max = round(max(xdata), 1)
        xdata_min = round(min(xdata), 1)
        ydata = data_df["Transmission Efficiency"]

        # Specify plot style
        plt.style.use('ggplot')

        idx = np.asarray([i for i in range(len(data_df['Diameter Midpoint']))])
        title_nm = "PCVI Calibration Results: PF = " + str(data_df["PF"][0]) + ", AF = " + str(data_df["AF"][0]) + ", SF = " + str(data_df["SF"][0]) + ", IF = " + str(data_df["IF"][0]) + ", IN/SF = " + str(data_df["IF/SF"][0])

        plt.figure(figsize = (10, 6))
        plt.scatter(xdata,ydata)
        plt.title(title_nm)
        plt.xlabel(r'Median Midpoint Diameter $\mathrm{(\mu m)}$')
        plt.ylabel("Transmission Efficiency")
        plt.xticks(idx)
        plt.xlim(right = (xdata_max + 1))

        # Generate a figure label for the filename
        figure_label = date+"_PF"+str(data_df['PF'][0]).replace('.', '_')+"_AF"+str(data_df['AF'][0]).replace('.', '_')+"_D50_TE"+".png"

        figure_nm = export_plots + "D50/" + figure_label

        ##### CURVE FITTING #####

        # Define a sigmoid function
        def sigmoid(x, L ,x0, k, b):
            y = L / (1 + np.exp(-k*(x-x0)))+b
            return (y)

        # Define L
        L = max(data_df['Transmission Efficiency']) # The maximum transmission efficiency

        # Initial guess for curve_fit model
        p0 = [max(ydata), np.median(xdata), 1, min(ydata)]
        
        try :
            # Fit a sigmoid to the data to idenitfy the cut point
            # Dogbox method is the most accurate however produces an overflow warning due to machine precision going beyond capacity
            popt, pcov = curve_fit(sigmoid, xdata, ydata, p0, method = 'dogbox', absolute_sigma = 'True', maxfev = 1000)
            
        except RuntimeError:
            print("Error - curve_fit failed")

        # Generate 1000 evenly spaced values between 0 and the maximum midpoint
        # y values are the corresponding model results
        x = np.linspace(xdata_min, xdata_max, 1000)
        y = sigmoid(x, *popt)

        # Plot best fit curve
        plt.plot(x, y, color = 'black', linestyle = '--')

        # Calculate sigma values
        sigma = np.sqrt(np.diagonal(pcov))

        # Extract the cut-point
        D50 = popt[1]
        D50 = round(D50, 2)

        # Extract the standard deviation corresponding to the cut point
        D50_sigma = sigma[1]
        D50_sigma = round(D50_sigma, 6)

        plt.plot(D50, (max(y)/2), 'x', color = 'blue')

        D50_label = r'Calculated $\mathrm{D_{50} = }$ ' + str(D50) + r' $\mathrm{\pm}$ ' + str(D50_sigma)

        plt.legend(["Best Fit Curve", D50_label, "Transmission Efficiencies"])

        # Save the figure
        plt.savefig(figure_nm, format = 'png', dpi = 'figure')
        
        plt.close()
        

Error - curve_fit failed
Error - curve_fit failed
Error - curve_fit failed
Error - curve_fit failed
Error - curve_fit failed
Error - curve_fit failed
Error - curve_fit failed
Error - curve_fit failed
Error - curve_fit failed
Error - curve_fit failed


In [79]:

# Remove NaN's to prevent errors in curve fitting
data_df = data_df.dropna(thresh = 10)

# Specify variables used in plotting
xdata = data_df["Diameter Midpoint"]
xdata_max = round(max(xdata), 1)
ydata = data_df["Transmission Efficiency (%)"]

# Specify plot style
plt.style.use('ggplot')

idx = np.asarray([i for i in range(len(data_df['Diameter Midpoint']))])
title_nm = "PCVI Calibration Results: PF = " + str(data_df["PF"][0]) + ", AF = " + str(data_df["AF"][0]) + ", SF = " + str(data_df["SF"][0]) + ", IF = " + str(data_df["IF"][0]) + ", IN/SF = " + str(data_df["IF/SF"][0])

plt.figure(figsize = (10, 6))
plt.scatter(xdata,ydata)
plt.title(title_nm)
plt.xlabel(r'Median Midpoint Diameter $\mathrm{(\mu m)}$')
plt.ylabel("Transmission Efficiency (%)")
plt.xticks(idx)
plt.xlim(right = (xdata_max + 1))


# Generate a figure label for the filename
figure_label = date+"_PF"+str(data_df['PF'][0]).replace('.', '_')+"_AF"+str(data_df['AF'][0]).replace('.', '_')+"_D50_TE"+".png"

figure_nm = export_plots + "D50/" + figure_label

##### CURVE FITTING #####

# Define a sigmoid function
def sigmoid(x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0)))+b
    return (y)

# Define L
L = max(data_df['Transmission Efficiency (%)']) # The maximum transmission efficiency

# Initial guess for curve_fit model
p0 = [max(ydata), np.median(xdata),1,min(ydata)]

# Fit a sigmoid to the data to idenitfy the cut point
# Dogbox method is the most accurate however produces an overflow warning due to machine precision going beyond capacity
popt, pcov = curve_fit(sigmoid, xdata, ydata, p0, method = 'dogbox', absolute_sigma = 'True')

# Generate 1000 evenly spaced values between 0 and the maximum midpoint
# y values are the corresponding model results
x = np.linspace(0, xdata_max, 1000)
y = sigmoid(x, *popt)

# Plot best fit curve
plt.plot(x, y, color = 'black', linestyle = '--')

# Calculate sigma values
sigma = np.sqrt(np.diagonal(pcov))

# Extract the cut-point
D50 = popt[1]
D50 = round(D50, 2)

# Extract the standard deviation corresponding to the cut point
D50_sigma = sigma[1]
D50_sigma = round(D50_sigma, 6)

plt.plot(D50, (max(y)/2), 'x', color = 'blue')

D50_label = r'Calculated $\mathrm{D_{50} = }$ ' + str(D50) + r' $\mathrm{\pm}$ ' + str(D50_sigma)

plt.legend(["Best Fit Curve", D50_label, "Transmission Efficiencies"])

# Save the figure
plt.savefig(figure_nm, format = 'png', dpi = 'figure')

plt.close()

